Script: clean_presidential_elections.ipynb
Author: Michel Gutmann
Purpose: Generates a state-election year dataset with presidential election results by party from David Leip's dataset.

In [5]:
import pandas as pd
import us
import os
import os.path as path
import numpy as np

In [6]:
parent_dir = os.path.abspath("/Users/revekkagershovich/Dropbox (MIT)/StateLaws")
os.chdir(parent_dir)
assert os.path.exists(parent_dir), "parent_dir does not exist"
data_dir = "./2_data/2_intermediate/political_data"
assert os.path.exists(data_dir), "Data directory does not exist"

In [7]:
input_file = "./2_data/1_raw/political_data/StateLevelData.xlsx"
assert os.path.exists(input_file), "Input file does not exist"
output_file = os.path.join(data_dir, "./statelevel_preselection_results.csv")
assert os.path.exists(output_file), "Output file does not exist"



In [75]:
##### DEFINE MAIN CLEANING FUNCTION #####
def clean_election(df, key):
    print(key)
    if key == "2008":
        df = df.drop(['State', 'Unnamed: 63'], axis=1)
    # Standardize format by keeping only raw result columns
    i = 0
    cols_to_keep = ["Unnamed: 0"]
    for col in df.columns:
        if i == 1:
            cols_to_keep.append(col)
        elif col == "% Total Vote":
            i = 1
    cols_to_keep = cols_to_keep[:-2]
    df_filtered = df[cols_to_keep].copy()
    df_filtered.rename({"Unnamed: 0": "state"}, axis=1, inplace=True)

    # Rename columns to appropriate names
    col_names = ["state"]
    for col in df_filtered.columns:
        col_str = str(col)
        if col_str != "state" and not col_str.startswith("Unnamed: "):
            last_name = "".join(col_str.lower().split(" "))
            if last_name not in ["democratic", "republican"]:
                last_name = "thirdparties"
            elif last_name == "democratic":
                last_name = "dem"
            elif last_name == "republican":
                last_name = "rep"
            col_names.append("votes_" + last_name)
        elif col_str != "state":
            col_names.append("pct_" + last_name)
    df_filtered.columns = col_names
    
    # Sum over third party results
    cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()
    return cleaned_df


In [58]:
##### LOAD DATA AND PUT IN FULL DATASET #####
# Load the Excel file with multiple sheets
dfs = pd.read_excel(input_file, header=1, sheet_name=None)

/opt/anaconda3/envs/Revekka_first_environment/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/opt/anaconda3/envs/Revekka_first_environment/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/opt/anaconda3/envs/Revekka_first_environment/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/opt/anaconda3/envs/Revekka_first_environment/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/opt/anaconda3/envs/Revekka_first_environment/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/opt/anaconda3/envs/Revekka_first_environment/lib/

In [74]:
df_2008 = dfs["2008"]
# df_2008 = df_2008.loc[:, ['State', 'Unnamed: 63']]
# print(df_2008.head())
df_2008 = df_2008.drop(['State', 'Unnamed: 63'], axis=1)
df_2008 = clean_election(df_2008, 200)

200


/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/1893037702.py:36: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()


In [63]:
df_2008.head()

,pct_dem,pct_rep,pct_thirdparties,state,votes_dem,votes_rep,votes_thirdparties
0,0.387404,0.603169,34.009427,Alabama,813479.0,1266546.0,19804.0
1,0.378894,0.594245,37.026861,Alaska,123594.0,193841.0,8767.0
2,0.449123,0.533940,35.016937,Arizona,1034707.0,1230111.0,46658.0
3,0.388647,0.587159,28.024194,Arkansas,422310.0,638017.0,34949.0
4,0.609436,0.369130,27.021434,California,8274473.0,5011781.0,303098.0


In [36]:
print(19804/(813479+1266546))

0.009521039410584008


In [69]:
df_2012 = dfs["2012"]
df_2012_clean = clean_election(dfs["2012"], 2012)

2012


/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/1298465466.py:35: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()


In [34]:
df_2012_clean.head()

,pct_dem,pct_rep,pct_thirdparties,state,votes_dem,votes_rep,votes_thirdparties
0,0.383590,0.605458,0.010951,Alabama,795696.0,1255925.0,22717.0
1,0.408127,0.548016,0.043858,Alaska,122640.0,164676.0,13179.0
2,0.444485,0.534846,0.020662,Arizona,1025232.0,1233654.0,47659.0
3,0.368790,0.605669,0.025541,Arkansas,394409.0,647744.0,27315.0
4,0.602390,0.371204,0.026407,California,7854285.0,4839958.0,344304.0


In [76]:
# Process each sheet (year) and combine them into a single DataFrame
i = 0
for key in dfs.keys():
    if key != "Copyright" and i == 0:
        print(f"Processing {key}")
        new_df = clean_election(dfs[key], key)
        new_df["year"] = int(key)
        full_df = new_df
        i += 1
    elif key != "Copyright":
        print(f"Processing {key}")
        new_df = clean_election(dfs[key], key)
        new_df["year"] = int(key)
        full_df = pd.concat([full_df, new_df], verify_integrity=True, axis=0, ignore_index=True)


Processing 2012
2012
Processing 2008
2008
Processing 2004
2004
Processing 2000
2000
Processing 1996
1996
Processing 1992
1992
Processing 1988
1988
Processing 1984
1984
Processing 1980
1980
Processing 1976
1976
Processing 1972
1972
Processing 1968
1968
Processing 1964
1964
Processing 1960
1960
Processing 1956
1956
Processing 1952
1952
Processing 1948
1948
Processing 1944
1944


/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/2347800667.py:36: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()
/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/2347800667.py:36: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()
/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/2347800667.py:36: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered.groupby(df_filtered.columns, axis=1).sum()
/var/folders/6h/20f1xzjj5xx3z0lxd2hqpjzc0000gp/T/ipykernel_12790/2347800667.py:36: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  cleaned_df = df_filtered

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [13]:
##### PREPARE AND OUTPUT FULL DATASET #####
# Remove asterisks from state names
full_df["state"] = full_df["state"].apply(lambda x: str(x)[:-1] if str(x).endswith("*") else str(x)) 

# Only keep rows corresponding to actual U.S. states
full_df = full_df.loc[
    full_df["state"].isin([state.name for state in us.STATES])
].copy()

# Add FIPS codes to the dataset
state_map = us.states.mapping("name", "fips")
full_df["statefips"] = full_df["state"].apply(lambda x: state_map[x])

# Save the cleaned dataset to the output file
# full_df.to_csv(output_file, index=False)
print(f"Dataset saved to {output_file}")

Dataset saved to ./2_data/2_intermediate/political_data/./statelevel_preselection_results.csv
